# M.0 Data initialization

In [1]:
import datetime

import time

import pyspark
import pyspark.sql.functions as f
from pyspark import SparkContext
from pyspark.sql import *
from pyspark.sql.types import *

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
NUMBER_OF_THREADS_TO_USE = "*"
spark = SparkSession.builder \
    .master('local[' + NUMBER_OF_THREADS_TO_USE + ']') \
    .appName('discret_pierre_signature_generation') \
    .config('spark.driver.memory', '200g') \
    .config('spark.driver.maxResultSize', '15g') \
    .config('spark.rapids.sql.enabled','true') \
    .getOrCreate()
sc = spark.sparkContext

spark

In [3]:
# those are the metadata used to sum the number of requests
meta = ['LocationId', 'MinuteWithinWeek']

# those are the metrics once we have gathered antennas and stuff
#metrics = ['Voice','SMS_3G','PS','CS','Call','SMS_4G','Service_Req','HO']
metrics = ['Call','SMS','Data','Mobility','Signalling','Emergency','Overload']

SourceParquetFilesLoc = '/WORKSPACE/Pierre/exports/ANRITSU_dataset/Paris'
MedianParquetFilesLoc = '/WORKSPACE/Pierre/exports/ANRITSU_meds/Paris'

ParquetFilesSignaturesLoc = '/WORKSPACE/Pierre/exports/ANRITSU_sigs/Paris'
ParquetFilesDistribsLoc = '/WORKSPACE/Pierre/exports/ANRITSU_distribs/Paris'

ParquetFilesALRsLoc = '/WORKSPACE/Pierre/exports/ANRITSU_ALR/Paris'
CsvFilesThresholdsLoc = '/WORKSPACE/Pierre/exports/ANRITSU_Paris_Thresholds.csv'


In [4]:
print(ParquetFilesSignaturesLoc)
print(ParquetFilesDistribsLoc)
print(ParquetFilesALRsLoc)
print(CsvFilesThresholdsLoc)

/WORKSPACE/Pierre/exports/ANRITSU_sigs/Paris
/WORKSPACE/Pierre/exports/ANRITSU_distribs/Paris
/WORKSPACE/Pierre/exports/ANRITSU_ALR/Paris
/WORKSPACE/Pierre/exports/ANRITSU_Paris_Thresholds.csv


## 0.1 Load and format data

In [5]:
MediansDFSP = spark.read.parquet(MedianParquetFilesLoc)

MediansDFSP.printSchema()

OriginalDataDFSP = spark.read.parquet(SourceParquetFilesLoc)

OriginalDataDFSP.printSchema()

#Medians = MediansDFSP.toPandas()

#MediansDFSP.show(10)

#print(Medians.info(verbose=True))

#lMinutes = []
#for i in range(0,7*24*60):
#    lMinutes.append([i])
#dfMinutes = sc.parallelize(lMinutes).toDF(["MinuteWithinWeek"])
#dfMinutes = dfMinutes.withColumn("MinuteWithinWeek", dfMinutes.MinuteWithinWeek.cast('integer'))


#MediansDF = pd.read_parquet(ParquetFilesLoc)



root
 |-- MinuteWithinWeek: integer (nullable = true)
 |-- Call: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- SMS: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- Data: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- Mobility: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- Signalling: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- Emergency: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- Overload: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- LocationId: integer (nullable = true)

root
 |-- time_local: timestamp (nullable = true)
 |-- Call: long (nullable = true)
 |-- SMS: long (nullable = true)
 |-- Data: long (nullable = true)
 |-- Mobility: long (nullable = true)
 |-- Signalling: long (nullable = true)
 |-- Emergency: long (nullable = true)
 |-- Overload: long (nullable = true)
 |-- Minute

In [22]:
LocIdsList = sorted([x.LocationId for x in MediansDFSP.select('LocationId').distinct().collect()])

print("found " + str(len(LocIdsList)) + " location groups")

#print(LocIdsList)

AllWeeksIdsList = sorted([x.WeekOfYear for x in OriginalDataDFSP.select('WeekOfYear').distinct().collect()])

print("All Weeks Ids:")
print(AllWeeksIdsList)

TestWeeksIdsList = range(17,22)

print("Test Weeks Ids:")
print(list(TestWeeksIdsList))

TrainWeeksIdsList = list(set(AllWeeksIdsList) - set(TestWeeksIdsList))

print("Train Weeks Ids:")
print(TrainWeeksIdsList)



found 878 location groups
All Weeks Ids:
[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
Train Weeks Ids:
[9, 10, 11, 12, 13, 14, 15, 16, 22, 23, 24]


In [24]:
TestWeeksIdsList = range(17,22)

print("Test Weeks Ids:")
print(list(TestWeeksIdsList))

TrainWeeksIdsList = list(set(AllWeeksIdsList) - set(TestWeeksIdsList))

print("Train Weeks Ids:")
print(TrainWeeksIdsList)


Test Weeks Ids:
[17, 18, 19, 20, 21]
Train Weeks Ids:
[9, 10, 11, 12, 13, 14, 15, 16, 22, 23, 24]


In [12]:


# select only the medians
for m in metrics:
    MediansDFSP = MediansDFSP.withColumn(m, MediansDFSP[m].getItem(1))
#MediansDFSP = MediansDFSP.select()



In [14]:
from tqdm import tqdm

from anr_discret import offlineMLbyPL

filteredDFs = []


#start_time = time.time()
with tqdm(total=len(LocIdsList), desc='Signature extraction') as pbar:
    # run 2 imbricated loops, this hasn't much consequence on the output thanks to the filter / partitioning thing 
    for LocId in LocIdsList:
        #if LocId>10:
        #    break
        # some minutes may be missing. Since they are already sorted, we use MinuteWithinWeek as an index 
        # converting the filtered data to pandas dataframe seems to be actually quite fast ; using the index is straitforward too
        df = MediansDFSP.filter(MediansDFSP.LocationId == LocId).toPandas().set_index('MinuteWithinWeek')

        # fill the missing minutes ; don't forget to set the proper value to columns weeksgroup and locationid
        df = df.reindex(range(0,24*7*60), fill_value=0).assign(LocationId=LocId)

        # now the DataFrame is ready to compute the butterworth filter
        filt = offlineMLbyPL.signature_filtered(df, cutoff=8, metrics=metrics).reset_index()
        # reset index because we want the "minutewithinweek" column back as a standard column

        # use the append function not to slow down computation because of the concatenation thing
        filteredDFs.append(filt)

        pbar.update(1)

            
# just concat everything at the same time
FilteredSignatureDF = pd.concat(filteredDFs)

# save as parquet because it's much more efficient - keep the same partition structure although it's not really good
# don't forget to remove the "index thing" (although we know that the actual index is ['WeeksGroup','LocationId', 'MinuteWithinWeek'])
#FilteredSignatureDF.to_parquet(path=ParquetFilesSignaturesLoc, partition_cols=['WeeksGroup','LocationId'], index=False)



Signature extraction: 100%|██████████| 878/878 [02:00<00:00,  7.32it/s]


In [15]:
FilteredSignatureDF.to_parquet(path=ParquetFilesSignaturesLoc, partition_cols=['LocationId'], index=False)

In [17]:
import scipy.stats as stats

SigmaSep = 2.32          # how do we select the set of absolute error values to 


def fit_distribution_eric_local(df:pd.DataFrame, metrics:list) -> dict:
    """Fit a Gamma distribution to the given columns the input dataframe.

    Parameters
    ----------
    df: pandas.core.frame.DataFrame
        The input dataframe (usually containing AE values)
    metrics: list
        The list of column names to fit the Gamma distribution

    Returns
    -------
    distrib: dict
        The dictionary containing, for each column name, the Gamma parameters
    """

    distrib = {}

    #compter nombre de cas où on est à np.inf pour vérifier que ça n'arrive pas
    df = df.replace([np.inf, -np.inf], np.nan)

    
    for m in metrics:
        # pour éviter sort : hypothèse distribution gaussienne + estimer à partir de mean et std
        # alpha = 2.32 std pour exclure 99% des donnée
        # alpha = 1.5 std pour exclure 95% données
        x_mean = df[m].mean()
        x_std = df[m].std()
        SeparationThresh = x_mean + SigmaSep * x_std
        x = df[m].loc[df[m] > SeparationThresh]
        
        # tout écart inférieur à seuil aura comme probabilité 1-proba
        proba = len(x.index) / len(df.index)
        
        # donc on stocke seuil et on compare dans la phase de détection à seuil
        # si value < seuil, on retourne 1-proba
        # sinon, on regarde la sf de la loi et on multiplie par proba
        # regarder la manière dont cette loi est respectée
        firsttuple = (SeparationThresh, proba, len(x.index))

        if len(x.index)>=3:
            # regarder options pour supprimer outliers dans la fonction gamma fit
            fittuple = stats.gamma.fit(x)
            # before the code used to translate this tuple used to be the following:

            distrib[m] = firsttuple + (fittuple[0], fittuple[-2], fittuple[-1])
        else:
            distrib[m] = firsttuple + (np.nan, np.nan, np.nan)
    return distrib








totalALR = []




MinALSThreshold = 1 / (60*24*365.25*10)




def compute_alr_eric_local(df:pd.DataFrame, distrib:dict, metrics:list) -> pd.DataFrame:
    """Computes the Anomaly Likelihood Rate (ALR) over the input dataframe.
    The ALR corresponds to the sum of the logs of the p-value for each service data.
    The p-value is obtained for each service data by fitting a Gamma distribution over the dataset.

    Parameters
    ----------
    df: pandas.core.frame.DataFrame
        The input dataframe (usually containing AE values)
    distrib: dict
        The error distribution parameters for all services
    metrics: list
        The list of column names corresponding to the service data

    Returns
    -------
    df: pandas.core.frame.DataFrame
        The same dataframe with the additional ALR column
    """

    res = pd.DataFrame().reindex_like(df)
    als = pd.DataFrame(index = res.index, columns = metrics)
    
    CopyCols = list(set(df.columns) - set(metrics))
    res[CopyCols] = df[CopyCols].copy(deep=True)

    for m in metrics:
        mLoc = df.columns.get_loc(m)
            
        SeparationThresh = distrib.loc['thresh',m]
        proba = distrib.loc['proba',m]
        nvalues = distrib.loc['nvalues',m]
        arg = distrib.loc['k',m]
        loc = distrib.loc['loc',m]
        scale = distrib.loc['theta',m]

        if nvalues<3:
            #print("case where the gamma law was not fitted on metric " + m)
            res.loc[:,m] = pd.Series(np.nan, index=df.index)
        else:
            # default value is 1-proba
            res.loc[:,m] = pd.Series((1. - proba), index=df.index)
            # the gamma law was fitted on this metric
            indexThresh = df.index[df[m]>SeparationThresh]

            res.loc[indexThresh,m] = (1. - proba) * stats.gamma.sf(df.loc[indexThresh, m], arg, loc=loc, scale=scale)
            res.loc[indexThresh,m].clip(lower=MinALSThreshold, inplace=True)

        als[m] = pd.Series(np.log(res[m]), index=res.index)

    res['ALR'] = als.sum(axis=1)
    return res






signatureDFs = []

ColumnsDropList = []
for m in metrics:
    ColumnsDropList.append(m+'_ref')

count = 0

distributionsDFList = []



#SmallerLocIdsList = LocIdsList[0:50]

    
with tqdm(total=len(LocIdsList), desc='Signature extraction') as pbar:
    # run 2 imbricated loops, this hasn't much consequence on the output thanks to the filter / partitioning thing 
    for LocId in LocIdsList:
        #if LocId>10:
        #    break
        # gather all the data corresponding to the training group and the corresponding location
        # start with the location
        #OriginalDataLocSP = OriginalDataDFSP.filter(OriginalDataDFSP.LocationId == LocId)


        #start_time = time.time()
        referenceDF = FilteredSignatureDF.loc[(FilteredSignatureDF['LocationId']==LocId)].drop(columns=['LocationId'])
        #print("grabbing the ref time performed in " + str((time.time() - start_time)) + " seconds")

        # then the corresponding weeks. I believe that it's more convenient by unioning stuff
        #start_time = time.time()

        #OriginalDataDFList = []
        ErrorsDFsList = []
        for wk in TrainWeeksIdsList:
            wkDF = OriginalDataDFSP.drop('time_local').filter((OriginalDataDFSP.WeekOfYear==wk) & (OriginalDataDFSP.LocationId == LocId)).toPandas()
            wkDF = wkDF.set_index('MinuteWithinWeek')
            wkDF = wkDF.reindex(range(0,24*7*60), fill_value=0).assign(WeekOfYear=wk, LocationId=LocId).fillna(0)

            for m in metrics:
                # 
                wkDF[m] = wkDF[m] - referenceDF[m]
                #wkDF[m] = abs(wkDF[m] - referenceDF[m])

            wkDF.reset_index(inplace=True)

            #print("length wkDF :" + str(len(wkDF.index)))

            ErrorsDFsList.append( wkDF )

        errors = pd.concat(ErrorsDFsList)


        #distributions = offlineMLbyPL.get_distrib_params(errors, meta=meta, metrics=metrics)
        distribution = pd.DataFrame(np.nan, index=['thresh', 'proba', 'nvalues', 'k','loc','theta'], columns=['LocationId', *metrics])
        distribution = distribution.assign(LocationId=LocId)
        distrib = fit_distribution_eric_local(errors, metrics)
        for m in metrics:
            for k in range(0,6):
                mLoc = distribution.columns.get_loc(m)
                distribution.iloc[k,mLoc] = distrib[m][k]


        distribution.index.names = ['GammaParam']

        #alrDF = compute_alr_eric_local(AbsErrors, GammaLawDF, metrics)
        alrDF = compute_alr_eric_local(errors, distribution, metrics)

        distribution.reset_index(inplace=True)
        distributionsDFList.append(distribution)


        totalALR.append(alrDF)

        #break


        pbar.update(1)
        
            
distributionsDF = pd.concat(distributionsDFList)
totalALRDF = pd.concat(totalALR)



#print(distributionsDF.info())
#print(distributionsDF.describe())
#print(distributionsDF)


#distributionsDF.to_parquet(path=ParquetFilesDistribsLoc, partition_cols=['WeeksGroup','LocationId'], index=False)



Signature extraction: 100%|██████████| 878/878 [34:41<00:00,  2.37s/it]


In [18]:
#ParquetFilesALRsLoc = '/WORKSPACE/Pierre/tests/' + DatasetPrefix + '_' + LocationPrefix + '_ALR0fill300LocIdsGroup012/'
distributionsDF.to_parquet(path=ParquetFilesDistribsLoc, partition_cols=['LocationId'], index=False)

#ParquetFilesDistribsLoc = '/WORKSPACE/Pierre/tests/' + DatasetPrefix + '_' + LocationPrefix + '_distribs300LocIdsGroup012/'
totalALRDF.to_parquet(path=ParquetFilesALRsLoc, partition_cols=['LocationId'], index=False)

In [19]:



def level1byPL(alr:pd.Series) -> float:
    """Sets a level 1 ALR threshold (i.e. pre-alert threshold) for the input data series.
    The threshold corresponds to the 2-sigma quantile of the ALR distribution.

    Parameters
    ----------
    alr: pandas.core.frame.Series
        The series of ALR values
    
    Returns
    -------
    thresh: float
        The ALR threshold of the series
    """

    #res = alr.replace([np.inf, -np.inf], np.nan).dropna(how="all")
    # 1 alert every 22 minutes was the parameter used by 
    # once per 4 hours -> 1 per 4*60 minutes
    #return res.quantile(1/240)
    return alr.quantile(0.00416667)

def level2byPL(alr:pd.Series) -> float:
    # 1 alert every 370 minutes
    """Sets a level 2 ALR threshold (i.e. alert threshold) for the input data series.
    The threshold corresponds to the 3-sigma quantile of the ALR distribution.

    Parameters
    ----------
    alr: pandas.core.frame.Series
        The series of ALR values
    
    Returns
    -------
    thresh: float
        The ALR threshold of the series
    """

    #res = alr.replace([np.inf, -np.inf], np.nan).dropna(how="all")
    #return res.quantile(1-0.9973)
    # once per day -> 1 per 24*60 minutes
    #return res.quantile(1/1440)
    return alr.quantile(0.0006944)

def level3byPL(alr:pd.Series) -> float:
    """Sets a level 3 ALR threshold (i.e. maximal alert threshold) for the input data series.
    The threshold corresponds to the 4-sigma quantile of the ALR distribution.

    Parameters
    ----------
    alr: pandas.core.frame.Series
        The series of ALR values
    
    Returns
    -------
    thresh: float
        The ALR threshold of the series
    """
    #1 alert every 15873 minute
    #res = alr.replace([np.inf, -np.inf], np.nan).dropna(how="all")
    #return res.quantile(1-0.999937)
    return alr.quantile(0.0000992)




#CsvFilesThresholdsLoc = 'exports/' + DatasetPrefix + '_thresholds_0fill2groups_' + LocationPrefix + '.csv'


# apply the offlineML functions to aggregate thresholds for each antenna
thresholds = totalALRDF.groupby(['LocationId']).ALR.agg([level1byPL, level2byPL, level3byPL])
thresholds.reset_index(inplace=True)

# don't forget to store the index or the following will be annoying
#thresholds.reset_index(inplace=True)
#thresholds.to_csv(CsvFilesThresholdsLoc)


In [20]:
#CsvFilesThresholdsLoc = '/WORKSPACE/Pierre/tests/' + DatasetPrefix + '_' + LocationPrefix + '_thresholds_300LocIdsGroup012.csv'
thresholds.to_csv(CsvFilesThresholdsLoc, index=False)

In [21]:
print(thresholds.info(verbose=True))
thresholds.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   LocationId  878 non-null    int64  
 1   level1byPL  878 non-null    float64
 2   level2byPL  878 non-null    float64
 3   level3byPL  878 non-null    float64
dtypes: float64(3), int64(1)
memory usage: 27.6 KB
None


,LocationId,level1byPL,level2byPL,level3byPL
count,878.000000,878.000000,878.000000,878.000000
mean,14191.920273,-11.290576,-22.958969,-inf
std,8216.132209,14.546115,34.008392,NaN
min,99.000000,-195.910663,-611.452600,-inf
25%,6773.250000,-11.136866,-22.593435,-37.063879
50%,14359.000000,-7.439725,-14.464153,-24.246616
75%,21358.500000,-5.727931,-10.979496,-17.432521
max,27972.000000,-0.002707,-1.196029,-3.059375


In [10]:
#print(LocIdsList)
# some verifications about what we have in the 

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

distributionsDF[distributionsDF['GammaParam']=='theta'].info()

#print("threshold statistics")
#distributionsDF[distributionsDF['GammaParam']=='thresh'].drop(columns=['LocationId', 'WeeksGroup']).describe()

print("\n\nproba statistics")
print(distributionsDF[distributionsDF['GammaParam']=='proba'].drop(columns=['LocationId', 'WeeksGroup']).describe())

LocIdsInfosDF = pd.read_csv('exports/Cancan_Paris_LocInfos_AllData.csv')
#print(LocIdsInfosDF.info(verbose=True))

SelectedLocIdsInfos = LocIdsInfosDF[LocIdsInfosDF['LocationId'].isin(SmallerLocIdsList)]
#print(SelectedLocIdsInfos.info(verbose=True))

print("\n\npossible non-zero values depending on the technology")
CountTechno = SelectedLocIdsInfos.groupby(['LocationId', 'TECHNO']).size().groupby('TECHNO').count()
print(CountTechno)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 5 to 5
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   GammaParam   50 non-null     object 
 1   LocationId   50 non-null     int64  
 2   WeeksGroup   50 non-null     int64  
 3   Voice        50 non-null     float64
 4   SMS_3G       50 non-null     float64
 5   PS           50 non-null     float64
 6   CS           50 non-null     float64
 7   Call         25 non-null     float64
 8   SMS_4G       25 non-null     float64
 9   Service_Req  25 non-null     float64
 10  HO           25 non-null     float64
dtypes: float64(8), int64(2), object(1)
memory usage: 4.7+ KB


proba statistics
           Voice     SMS_3G         PS         CS       Call     SMS_4G  Service_Req         HO
count  50.000000  50.000000  50.000000  50.000000  50.000000  50.000000    50.000000  50.000000
mean    0.037998   0.031231   0.025220   0.023378   0.018931   0.016615     0.0

In [90]:
thresholds.describe()


,LocationId,WeeksGroup,level1byPL,level2byPL,level3byPL
count,300.000000,300.0,300.000000,300.000000,300.000000
mean,1061.803333,0.0,-5.195179,-11.174876,-18.086576
std,110.215135,0.0,3.778842,6.997304,9.384837
min,875.000000,0.0,-26.323358,-56.610089,-65.210885
25%,961.750000,0.0,-6.574128,-14.145312,-22.871932
50%,1064.000000,0.0,-4.398969,-9.676692,-16.536602
75%,1156.500000,0.0,-2.668047,-6.774801,-11.970358
max,1251.000000,0.0,-0.000121,-0.000121,-0.000123
